In [3]:
import os

In [1]:
%pwd

'c:\\Users\\rites\\github\\TextSummarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\rites\\github\\TextSummarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [11]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size



In [25]:
class DataImgestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded successfully with folowing info: {headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 

    def extract_zip_file(self):
        """
        Extracts the zip file to a folder"""

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    

[2023-06-06 16:31:22,932: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-06 16:31:22,934: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-06 16:31:22,935: INFO: common: created directory at: artifacts]
[2023-06-06 16:31:22,936: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-06 16:31:23,662: INFO: 135721713: artifacts/data_ingestion/data.zip downloaded successfully with folowing info: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3508:7BDC:F6DB4:120C44:647F979D
Accept-Ranges: bytes
Date: Tue, 06 Jun 2023 20:31:25 GMT
Via: 1.1 varnish
X-Served-By: cache-yyz4549-YYZ
X-C